In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

/Users/ludai/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
if __name__ == '__main__':
    base_path = "/Users/ludai/Desktop/Github/fda/Assignment3/src/"
    x_size = 0.5
    y_size = 0.5
    fw = [500, 1000, 5, 4, 1./22., 2, 8,1] #feature weights
    #fw = [1, 1, 1, 1, 1, 1, 1]
    th = 5 #Keeping place_ids with more than th samples.
    accuracy = np.zeros(3, dtype=np.float)
    for index in range(1):
        string  =  'subset%s.csv' % 2
        data_path = base_path + string
        #print data_path
        df = pd.read_csv(data_path, usecols=['row_id','x','y','accuracy','time','place_id'], index_col = 0)
        initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
        d_times = pd.DatetimeIndex(initial_date + np.timedelta64(t, 'm') for t in df.time.values)
        #get info from time field
        df['hour'] = d_times.hour * fw[2]
        df['weekday'] = d_times.weekday * fw[3]
        df['day'] = (d_times.dayofyear * fw[4]).astype(int)
        df['month'] = d_times.month * fw[5]
        df['year'] = (d_times.year - 2013) * fw[6]
        #df = df.drop(['time'], axis=1) 
        df.x = df.x.values * fw[0]
        df.y = df.y.values * fw[1]
        df.accuracy = df.accuracy * fw[7]
        print np.shape(df)
        place_counts = df.place_id.value_counts()
        print np.shape(place_counts)
        #drop rows with place_id which has samples < th
        mask = (place_counts[df.place_id.values] >= th).values
        print np.shape(mask)
        df = df.loc[mask]
        
        #split data into training and testing sets
        #df_train = df.sample(frac=0.7)
        #df_test = df.loc[~df.index.isin(df_train.index)]
        df_test = df[df['time'] > 600000].sample(n=200000)
        df_train = df.loc[~df.index.isin(df_test.index)]
        
        print np.shape(df), np.shape(df_train), np.shape(df_test)
        
        #preparing training sets
        le = preprocessing.LabelEncoder()
        train_Y = le.fit_transform(df_train['place_id'].values)
        #print train_Y
        train_X = df_train.drop(['place_id','time'], axis=1).values.astype(int)
        #print train_X
     
        knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance', metric='manhattan')
        knn_clf.fit(train_X, train_Y)
        
        #preparing testing sets
        truth = df_test['place_id'].values
        test  =  df_test.drop(['place_id','time'], axis =1 ).values.astype(int)
        #print np.shape(test_place), np.shape(test)
    
        knn_pred = knn_clf.predict(test)
        print knn_pred
        pred = le.inverse_transform(knn_pred)
        print pred
        print np.shape(pred), np.shape(truth)
        #count = 0
        #total = np.shape(labels)[0]
        correct = (pred == truth)
#         for i in range(total):
#             if labels[i]==test_place[i]:
#                 count+=1
        #print count, float(count)/float(total)
        #accuracy[index] = (float(count)/float(total))
        print correct

(1188294, 10)
(17039,)
(1188294,)
(1172692, 10) (972692, 10) (200000, 10)


In [72]:
     print accuracy

[ 0.5432512   0.39670519  0.39515679]


In [34]:
pred = le.inverse_transform(knn_pred)
print pred
print np.shape(pred), np.shape(truth)
#count = 0
#total = np.shape(labels)[0]
correct = (pred == truth)
#         for i in range(total):
#             if labels[i]==test_place[i]:
#                 count+=1
#print count, float(count)/float(total)
#accuracy[index] = (float(count)/float(total))
print correct
print np.mean(correct)

[2325062540 1783141247 6684601529 ..., 6315361820 3305662426 5215276306]
(200000,) (200000,)
[ True  True False ...,  True  True  True]
0.47611
